In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]
data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
#data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

# Specify the columns you want to standardize
columns_to_standardize = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]

# Create a StandardScaler object
scaler = StandardScaler()

# Loop through the columns and standardize each one
for column in columns_to_standardize:
    data[column] = scaler.fit_transform(data[[column]])



import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-07-31
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-07-31')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data['date'])

# Extract day, month, and year from the date column
data['day'] = data2['date'].dt.day
data['month'] = data2['date'].dt.month
data['year'] = data2['date'].dt.year

# Define the embedding dimensions
embedding_dim = 1  # You can adjust this dimension as needed

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor(data['day'].values)
month_tensor = torch.LongTensor(data['month'].values)
year_tensor = torch.LongTensor(data['year'].values - 1980)  # Convert years to an index from 0 to 43


# Pass tensors through embedding layers to get embeddings
day_embeddings = day_embedding(day_tensor)
month_embeddings = month_embedding(month_tensor)
year_embeddings = year_embedding(year_tensor)

# Concatenate the embeddings
date_embeddings = torch.cat((day_embeddings, month_embeddings, year_embeddings), dim=1)

# Print the resulting embeddings
print(date_embeddings)









tensor([[ 0.5248, -0.2405, -0.4801],
        [-2.3531, -0.2405, -0.4801],
        [ 0.9835, -0.2405, -0.4801],
        ...,
        [ 1.3984,  0.3891,  1.0989],
        [ 0.1517,  0.3891,  1.0989],
        [-0.8863,  0.3891,  1.0989]], grad_fn=<CatBackward0>)


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)



# Concatenate date embeddings with your training data
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)

# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)




# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        # Define the LSTM layer as a class attribute
        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out




# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 16
n_layers = 4
sequence_length = 65  # Keep this as 1 for your input data
batch_size = 64



# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00015
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')


# Training loop
num_epochs = 4500

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Extract features on the testing set
        model.eval()
        with torch.no_grad():
            # Extract features from the hidden states
            hidden_states, _ = model.lstm(x_test_tensor.view(x_test_tensor.size(0), 1, -1))
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
        # You can now use 'hidden_states' as the feature representations of your sequences
        # The shape of 'hidden_states' will be (batch_size, sequence_length, hidden_dim)

# Print hidden_states after training
print(hidden_states)










In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Leer el archivo CSV
data = pd.read_csv("../data/data/aapl_raw_data.csv")

# Preprocesamiento de datos
data = data.fillna(0)

columns_to_standardize = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]
scaler = MinMaxScaler()
for column in columns_to_standardize:
    data[column] = scaler.fit_transform(data[[column]])

# ... Aquí puedes agregar más procesamiento de datos si es necesario

# Separar datos en características (x) y objetivo (y)
x = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y = data['close'].values

# Dividir datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=45)

# Convertir a tensores de PyTorch
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Modelo LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Configurar modelo y entrenamiento
input_size = 7
hidden_size = 16
output_size = 1
model = LSTMModel(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento (este es un ejemplo básico, puedes mejorar esto)
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(x_train_tensor.view(-1, 1, input_size))
    loss = criterion(outputs, y_train_tensor.view(-1, 1))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

# Hacer una predicción
model.eval()
with torch.no_grad():
    test_output = model(x_test_tensor.view(-1, 1, input_size))
    predicted = scaler.inverse_transform(test_output.numpy())  # Si deseas revertir la escala

print("Predicted values:", predicted)


Epoch [10/100], Loss: 29920.3848
Epoch [20/100], Loss: 29912.7090
Epoch [30/100], Loss: 29904.5117
Epoch [40/100], Loss: 29895.3555
Epoch [50/100], Loss: 29884.7285
Epoch [60/100], Loss: 29872.1465
Epoch [70/100], Loss: 29857.1953
Epoch [80/100], Loss: 29839.5410
Epoch [90/100], Loss: 29818.9609
Epoch [100/100], Loss: 29795.3145
Predicted values: [[7.3902086e+08]
 [8.6336608e+08]
 [7.8182221e+08]
 ...
 [7.5393997e+08]
 [7.4635821e+08]
 [7.1324435e+08]]


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming you have loaded your data and performed preprocessing steps (as previously shared)

# Set up the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Assuming you have your x_train and y_train from your data
# ... (preprocessing and splitting code here)

input_size = 7
hidden_dim = 16
output_size = 1

model = LSTMModel(input_size, hidden_dim, output_size)

# Assuming you've trained your model (add your training loop and logic here)
# ... (training code here)

# Now, for a single prediction:
# Make sure x_test_tensor is the tensor you want to predict from (after preprocessing and shaping)
single_input = x_test_tensor[0].view(1, 1, input_size)  # Reshape for single prediction

# Set the model to evaluation mode
model.eval()

# Perform the prediction
with torch.no_grad():
    prediction = model(single_input)

# Extract and print the predicted value
predicted_value = prediction[0][0].item()
print(f"The predicted value is: {predicted_value}")


The predicted value is: 0.09357930719852448
